# Scraping Youtube Data for Top 50 Steven Wilson Videos
#### by Hrishikesh Dutta 

In [1]:
#url scraping from youtube search
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

opt = Options()
opt.add_argument("--incognito")

wd=r"C:\Users\Mahe\Desktop\Work\Youtube Scraping\chromedriver\chromedriver.exe"
driver = webdriver.Chrome(executable_path=wd, options=opt)
driver.get("http://youtube.com")

#searching for Steven Wilson videos
driver.find_element_by_name("search_query").send_keys("Steven Wilson")
driver.find_element_by_id("search-icon-legacy").click()

wait = WebDriverWait(driver, 10)
for i in range(10):
    wait.until(EC.visibility_of_element_located((By.TAG_NAME,"body"))).send_keys(Keys.END)
    time.sleep(3)
    
WebDriverWait(driver, 30).until(EC.presence_of_all_elements_located((By.ID, "video-title")))
links = driver.find_elements_by_id("video-title")
urls=[]
for x in links:
    urls.append(x.get_attribute("href"))

In [2]:
#cleaning the url data
import pandas as pd

df = pd.DataFrame(urls)
df_clean=df.dropna(how='any')
df_clean.columns=['urls']
df_clean.reset_index(drop=True, inplace=True)
df_clean.head(50)

,urls
0,https://www.youtube.com/watch?v=OzglcGQHUKE
1,https://www.youtube.com/watch?v=qFwcahcDzP4
2,https://www.youtube.com/watch?v=ycYewhiaVBk
3,https://www.youtube.com/watch?v=cNTaFArEObU
4,https://www.youtube.com/watch?v=O_H0NO-Cyew
5,https://www.youtube.com/watch?v=sh5mWzKlhQY
6,https://www.youtube.com/watch?v=WbWhpfXisZw
7,https://www.youtube.com/watch?v=nTjc1sPktlY
8,https://www.youtube.com/watch?v=_nKN9_H1Dks
9,https://www.youtube.com/watch?v=qf4ejxFXTRU


In [3]:
#youtube data scraping
from bs4 import BeautifulSoup 
import requests 

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'}
#function to scrape necessary data from youtube
def scrape_data(url):
    req = requests.get(url, headers=headers)
    s = BeautifulSoup(req.text, "html.parser")
    title = s.find("span", class_="watch-title").get_text(strip=True)
    date = s.find(class_ = 'watch-time-text').text.replace('Published on ', '').replace('Premiered on ', '')
    views = int(s.find("div", attrs={"class": "watch-view-count"}).text[:-6].replace(",", ""))
    likes = int(s.find("button", attrs={"title": "I like this"}).text.replace(",", ""))
    dislikes = int(s.find("button", attrs={"title": "I dislike this"}).text.replace(",", ""))
    data = {'video_link':url, 'title':title, 'views':views, 'uploaded_date':date, 'likes':likes, 'dislikes':dislikes}
    return data

In [5]:
#loop to collect data for top 50 urls
all_data = []
urls=df_clean['urls'].head(50).values.tolist()
for url in urls:
    all_data.append(scrape_data(url))
#printing and saving the data to csv file
df = pd.DataFrame(all_data)
df.to_csv('data.csv')

In [6]:
df

,video_link,title,views,uploaded_date,likes,dislikes
0,https://www.youtube.com/watch?v=OzglcGQHUKE,Steven Wilson - Stoneage Dinosaurs (Cardiacs C...,20322,22 Jul 2020,1990,25
1,https://www.youtube.com/watch?v=qFwcahcDzP4,Steven Wilson - Ancestral (from Home Invasion:...,120684,14 May 2020,5585,32
2,https://www.youtube.com/watch?v=ycYewhiaVBk,Steven Wilson - Drive Home,6564391,20 Sep 2013,67291,907
3,https://www.youtube.com/watch?v=cNTaFArEObU,Steven Wilson - Pariah ft. Ninet Tayeb,4146061,18 May 2017,43235,1082
4,https://www.youtube.com/watch?v=O_H0NO-Cyew,STEVEN WILSON - PERSONAL SHOPPER (OFFICIAL AUDIO),523062,12 Mar 2020,13218,955
5,https://www.youtube.com/watch?v=sh5mWzKlhQY,Steven Wilson - Routine,3948601,29 Oct 2015,49661,698
6,https://www.youtube.com/watch?v=WbWhpfXisZw,Porcupine Tree Arriving somewhere but not here,6219330,14 Feb 2011,42252,1065
7,https://www.youtube.com/watch?v=nTjc1sPktlY,Lazarus- Porcupine Tree (Lyrics),4515034,7 Feb 2011,35254,649
8,https://www.youtube.com/watch?v=_nKN9_H1Dks,Porcupine Tree - Russia on Ice,2068994,16 Nov 2012,16705,304
9,https://www.youtube.com/watch?v=qf4ejxFXTRU,Steven Wilson - Drive Home (from Drive Home),115750,23 Sep 2013,1102,11
